In [1]:
from common_functions import ret_metabase, google_sheets, upload_dataframe_to_snowflake, snowflake_query, upload_dataframe_to_pg, dwh_query, initialize_env
import pandas as pd
from datetime import datetime, timedelta
import datetime as dt
import time
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
import requests
from pathlib import Path
from io import StringIO
import pandas as pd
# import psycopg2
import numpy as np
import gspread
import pytz

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


ModuleNotFoundError: No module named 'gspread'

In [2]:
df = pd.read_excel('Coupon_testing.xlsx')
df

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,RETAILER_ID,ACTIVE_SELLERS,DAYS_INACTIVE,NMV_LAST_45,NMV,ORDER_COUNT_LAST_45,ORDER_COUNT,random,Counter,group_type
0,147884,27,11,11937.00000,327817.134038,6,97,0.000006,0,C1
1,349281,24,15,10405.00000,237968.068788,4,108,0.000148,1,C1
2,714314,26,43,1711.75000,24631.277662,1,10,0.000579,2,C1
3,360479,27,9,9724.75000,106049.359650,5,70,0.000708,3,C1
4,268508,37,17,36717.94000,548429.627536,5,92,0.000985,4,C1
...,...,...,...,...,...,...,...,...,...,...
4932,606793,32,27,7904.50000,27819.459000,3,17,0.998693,4932,T4
4933,286448,32,28,2106.55000,158641.149620,1,62,0.998869,4933,T4
4934,926882,7,28,1799.96875,1799.968750,1,1,0.998990,4934,T4
4935,475396,38,22,7372.50000,47202.867800,2,27,0.999208,4935,T4


In [6]:
coupon_data = df[df['group_type'].isin(['T3', 'T4'])]
coupon_data

,RETAILER_ID,ACTIVE_SELLERS,DAYS_INACTIVE,NMV_LAST_45,NMV,ORDER_COUNT_LAST_45,ORDER_COUNT,random,Counter,group_type
2906,594978,16,19,1296.000000,15344.081200,1,12,0.600025,2906,T3
2907,619170,17,8,20104.019333,192284.997733,7,58,0.600333,2907,T3
2908,17311,35,11,11914.350000,328468.518870,3,105,0.600390,2908,T3
2909,245132,39,8,4757.750000,63113.580308,3,47,0.600758,2909,T3
2910,63547,24,16,5073.250000,347440.177887,1,135,0.600791,2910,T3
...,...,...,...,...,...,...,...,...,...,...
4932,606793,32,27,7904.500000,27819.459000,3,17,0.998693,4932,T4
4933,286448,32,28,2106.550000,158641.149620,1,62,0.998869,4933,T4
4934,926882,7,28,1799.968750,1799.968750,1,1,0.998990,4934,T4
4935,475396,38,22,7372.500000,47202.867800,2,27,0.999208,4935,T4


In [14]:
coupon_100 = df[df['group_type'] == 'T4']
coupon_100

,RETAILER_ID,ACTIVE_SELLERS,DAYS_INACTIVE,NMV_LAST_45,NMV,ORDER_COUNT_LAST_45,ORDER_COUNT,random,Counter,group_type,user_id
3933,804658,18,15,17240.416667,35055.350000,2,4,0.800081,3933,T4,NaN
3934,698374,16,13,14847.400000,49565.610000,5,26,0.800593,3934,T4,NaN
3935,226129,12,11,27563.918800,267961.174550,6,77,0.800724,3935,T4,NaN
3936,19818,26,8,7384.000000,52413.600000,3,5,0.801176,3936,T4,NaN
3937,535444,30,20,1622.350000,60337.579400,1,16,0.801182,3937,T4,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4932,606793,32,27,7904.500000,27819.459000,3,17,0.998693,4932,T4,NaN
4933,286448,32,28,2106.550000,158641.149620,1,62,0.998869,4933,T4,NaN
4934,926882,7,28,1799.968750,1799.968750,1,1,0.998990,4934,T4,NaN
4935,475396,38,22,7372.500000,47202.867800,2,27,0.999208,4935,T4,NaN


In [19]:

from datetime import datetime, timedelta
# Replace with your WonderPush API key
API_KEY = "YjFkMmExOWFlN2IxNzQzYzFlMzA4YThhNmE4ZDgwZmFiNmFlNTM2ODVhODhjMzM1NjA3NWU0MWI3M2YxYTZmYw"


df['user_id'] = 'EG_retailers_' + coupon_50['RETAILER_ID'].astype(str)
try:
    # Read the Excel file into a pandas DataFram
 
    for user_id, group in df.groupby("user_id"):
        

        # Build the PATCH body: only the field you want to change under "custom"
        patch_body = {
            "custom": {
                "string_send_to": "Yes_for_coupons",
                "string_coupon_ammount": "50-coupon" 
            }
        }

        # Send a PATCH to /v1/users/{userId}
        url = (
            f"https://management-api.wonderpush.com/v1/users/{user_id}"
            f"?accessToken={API_KEY}"
        )
        headers = {"Content-Type": "application/json"}
        response = requests.patch(url, headers=headers, json=patch_body)

        if response.status_code in [200, 201, 202]:
            print(f"✅ Patched string_mpproducts for user {user_id}")
        else:
            print(f"❌ Error for {user_id}: {response.status_code} – {response.text}")

except ValueError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")     


✅ Patched string_mpproducts for user EG_retailers_100360
✅ Patched string_mpproducts for user EG_retailers_102047
✅ Patched string_mpproducts for user EG_retailers_10213
✅ Patched string_mpproducts for user EG_retailers_102798
✅ Patched string_mpproducts for user EG_retailers_10331
✅ Patched string_mpproducts for user EG_retailers_103485
✅ Patched string_mpproducts for user EG_retailers_103543
✅ Patched string_mpproducts for user EG_retailers_10379
✅ Patched string_mpproducts for user EG_retailers_103807
✅ Patched string_mpproducts for user EG_retailers_10392
✅ Patched string_mpproducts for user EG_retailers_104095
✅ Patched string_mpproducts for user EG_retailers_104170
✅ Patched string_mpproducts for user EG_retailers_105310
✅ Patched string_mpproducts for user EG_retailers_105369
✅ Patched string_mpproducts for user EG_retailers_106585
✅ Patched string_mpproducts for user EG_retailers_108624
✅ Patched string_mpproducts for user EG_retailers_109201
✅ Patched string_mpproducts for use

In [17]:

from datetime import datetime, timedelta
# Replace with your WonderPush API key
API_KEY = "YjFkMmExOWFlN2IxNzQzYzFlMzA4YThhNmE4ZDgwZmFiNmFlNTM2ODVhODhjMzM1NjA3NWU0MWI3M2YxYTZmYw"


user_id = 'EG_retailers_299600'




# Build the PATCH body: only the field you want to change under "custom"
patch_body = {
    "custom": {

        "string_coupon_ammount": "100-coupon"
    }
}

# Send a PATCH to /v1/users/{userId}
url = (
    f"https://management-api.wonderpush.com/v1/users/{user_id}"
    f"?accessToken={API_KEY}"
)
headers = {"Content-Type": "application/json"}
response = requests.patch(url, headers=headers, json=patch_body)

if response.status_code in  [200, 201, 202]:
    print(f"✅ Patched string_mpproducts for user {user_id}")
else:
    print(f"❌ Error for {user_id}: {response.status_code} – {response.text}")

    

✅ Patched string_mpproducts for user EG_retailers_299600


In [22]:
df = pd.read_excel('Coupon_testing.xlsx')
df


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,RETAILER_ID,ACTIVE_SELLERS,DAYS_INACTIVE,NMV_LAST_45,NMV,ORDER_COUNT_LAST_45,ORDER_COUNT,random,Counter,group_type
0,147884,27,11,11937.00000,327817.134038,6,97,0.000006,0,C1
1,349281,24,15,10405.00000,237968.068788,4,108,0.000148,1,C1
2,714314,26,43,1711.75000,24631.277662,1,10,0.000579,2,C1
3,360479,27,9,9724.75000,106049.359650,5,70,0.000708,3,C1
4,268508,37,17,36717.94000,548429.627536,5,92,0.000985,4,C1
...,...,...,...,...,...,...,...,...,...,...
4932,606793,32,27,7904.50000,27819.459000,3,17,0.998693,4932,T4
4933,286448,32,28,2106.55000,158641.149620,1,62,0.998869,4933,T4
4934,926882,7,28,1799.96875,1799.968750,1,1,0.998990,4934,T4
4935,475396,38,22,7372.50000,47202.867800,2,27,0.999208,4935,T4


In [24]:
df = df[df['group_type'].isin(['T1', 'T2'])]
df['coupon_value'] = df['group_type'].map({'T1': 50, 'T2': 100})
df

,RETAILER_ID,ACTIVE_SELLERS,DAYS_INACTIVE,NMV_LAST_45,NMV,ORDER_COUNT_LAST_45,ORDER_COUNT,random,Counter,group_type,coupon_value
953,507888,28,37,3191.00,31692.572500,1,7,0.200703,953,T1,50
954,614553,44,17,245722.50,390298.750000,7,18,0.200973,954,T1,50
955,531975,27,25,3653.33,42113.701000,1,17,0.200983,955,T1,50
956,676445,15,8,16815.28,304123.185659,5,88,0.201091,956,T1,50
957,316327,24,19,4675.50,104593.934867,2,79,0.201159,957,T1,50
...,...,...,...,...,...,...,...,...,...,...,...
2901,92878,17,37,1367.50,157831.777179,1,116,0.598486,2901,T2,100
2902,930550,26,31,1880.25,1880.250000,1,1,0.598545,2902,T2,100
2903,263680,7,22,3408.00,44841.656833,1,24,0.598809,2903,T2,100
2904,214822,7,31,490.00,743943.374766,1,252,0.598971,2904,T2,100


In [25]:

if df is None or df.empty:
    print("No rows found in Coupons sheet; nothing to send.")
    

# Ensure no duplicates for retailer_id and remove rows without an ID
duplicates_removed = 0
try:
    if 'RETAILER_ID' in df.columns:
        original_count = len(df)
        df = df.dropna(subset=['retailer_id'])
        df = df[df['retailer_id'].astype(str).str.len() > 0]
        df = df.drop_duplicates(subset=['RETAILER_ID'], keep='first')
        duplicates_removed = max(0, original_count - len(df))
    else:
        print("Column 'retailer_id' not found in sheet; aborting send.")
        
        
except Exception as e:
    print(f"Failed to clean duplicates in 'retailer_id': {e}")

sent = 0
failed = 0
last_status = None

for _, row in df.iterrows():
    user = row.get('RETAILER_ID')
    coupon_value = row.get('coupon_value')

    url = "https://management-api.wonderpush.com/v1/deliveries"
    querystring = {"accessToken": "YjFkMmExOWFlN2IxNzQzYzFlMzA4YThhNmE4ZDgwZmFiNmFlNTM2ODVhODhjMzM1NjA3NWU0MWI3M2YxYTZmYw"}
    payload = {
        "targetUserIds": [f"EG_retailers_{user}"],
        "notification": {
            "alert": {
                "text": f"💰 وفر {coupon_value} ج.م على طلبك الجاي! عندك كوبون جاهز للاستخدام 🎯 اطلب من تجار مكسب دلوقتي واستمتع بالخصم 🚀 ",
                "targetUrl": "https://maxab.page.link?deep_link=myApp://marketPlaceHomeFragment"
            }
        }
    }
    headers = {"Content-Type": "application/json"}

    try:
        response = requests.request("POST", url, json=payload, headers=headers, params=querystring)
        last_status = getattr(response, 'status_code', None)
        response_text = getattr(response, 'text', '')
        print(response_text)

        # Prefer explicit success flag in WonderPush response body
        success_flag = None
        try:
            response_json = response.json()
            if isinstance(response_json, dict):
                success_flag = response_json.get("success")
        except Exception:
            # Non-JSON or malformed response; fall back to HTTP status
            response_json = None

        if success_flag is True:
            sent += 1
        elif success_flag is False:
            failed += 1
        else:
            # Fallback when no explicit success flag exists: rely on HTTP status
            if response is not None and response.status_code in [200, 201]:
                sent += 1
            else:
                failed += 1
    except Exception as e:
        print(f"Failed to send notification for user {user}: {e}")
        failed += 1

Failed to clean duplicates in 'retailer_id': ['retailer_id']
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"notificationId":"@1","notificationIds":["@1"]}
{"success":true,"campaignId":null,"no